<a href="https://colab.research.google.com/github/ShengYu97/BT4222/blob/master/DataCollection/BT4222_Project_DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import

In [0]:
!pip install habanero

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import requests
from habanero import counts
import urllib
import json
import pandas as pd
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
%matplotlib inline

In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Ultility Tools

In [0]:
def listToItem(lst):
  if type(lst) is list:
    if len(lst) != 0:
      return lst[0]

In [0]:
def getDfByJournal(issn, nrows, offset=1):
  df = pd.DataFrame()
  for i in range(offset):
    url = "https://api.crossref.org/journals/"+issn+"/works?filter=has-abstract:true&rows="+str(nrows)+"&offset="+str(offset)
    headers={"Accept": "application/json"}
    req = requests.get(url, headers=headers)
    if req.status_code == 200:
      json_str = json.dumps(req.json()['message']['items'])
      df = df.append(pd.read_json(json_str))      
  return df

In [0]:
df = getDfByJournal('2046-2069', 10, 2)
df.shape

(20, 35)

In [0]:
url = "https://api.crossref.org/journals/"+'2046-2069'+"/works?filter=has-abstract:true&rows="+str(10)+"&offset="+str(2)
headers={"Accept": "application/json"}
req = requests.get(url, headers=headers)

In [0]:
df = pd.DataFrame()

In [0]:
json_str = json.dumps(req.json()['message']['items'])
df = df.append(pd.read_json(json_str))      

In [0]:
df

""


In [0]:
def cleandata(x):
    x = x.strip()
    x = x.replace('<jats:p>', '')
    x = x.replace('</jats:p>', '')
    text = x.split(' ')
    text = ' '.join([i for i in text if not i.isdigit()])
    text = re.sub("[!@#$+%*:()'-]", ' ', text)
    
    #remove numbers and convert into lowercase
    text = re.sub(r'\d+', '', text).lower()
    
    #remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    text = ' '.join([i for i in tokens if not i in stop_words])
    
    stemmer= PorterStemmer()
    words = text.split(' ')
    alltext = []
    for word in words:
        alltext.append(word)
    return ' '.join(alltext)


# def removeNumbers(x):
#     return re.sub(r'\d+', '', x)
    
def tolowercase(x):
    return x.lower()

def removePunctuation(x):
    translator = str.maketrans('', '', string.punctuation)
    return x.translate(translator)

##Convert container title from list to string
def listToItem(lst):
    if type(lst) == list:
        if len(lst) != 0:
            return lst[0]

#remove stop words
# def removeStopWords(x):
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(x)
#     result = ' '.join([i for i in tokens if not i in stop_words])
#     return result

#stemming words
# def stemmmingWords(x):
#     stemmer= PorterStemmer()
#     words = x.split(' ')
#     text = []
#     for word in words:
#         text.append(stemmer.stem(word))
#     return ' '.join(text)

# Get ISSN list from query keyword

### Fetching data by keywords

In [0]:
keyword = "thermal%20comfort"
nrows = 1000


#Fetching...
url = "http://api.crossref.org/works?query="+keyword+"&filter=has-abstract:true&sort=is-referenced-by-count&rows="+str(nrows) 
headers={"Accept": "application/json"}
req = requests.get(url, headers=headers)
json_str = json.dumps(req.json()['message']['items'])
df = pd.read_json(json_str)

### Selecting issn to include

In [0]:
df['container-title'] = df['container-title'].apply(lambda x: listToItem(x))
df['ISSN'] = df['ISSN'].apply(lambda x: listToItem(x))
## Do Look at the line for selecting which journals to include
print(df['container-title'].value_counts())
issnList = list(df['ISSN'].value_counts().keys())
len(issnList)

RSC Advances                                                                              117
Journal of Materials Chemistry A                                                           63
Physical Chemistry Chemical Physics                                                        43
Journal of Materials Chemistry C                                                           40
Australian Journal of Chemistry                                                            37
Journal of Glaciology                                                                      33
Nanoscale                                                                                  32
RSC Adv.                                                                                   25
Australian Journal of Physics                                                              23
Journal of Applied Crystallography                                                         22
J. Mater. Chem. A                                           

169

### Fetch by issn / journals

In [0]:
nrows = 1000
nissn = 12
offset = 10
df = pd.DataFrame()
for issn in issnList[:nissn]:
  print(issn)
  res = getDfByJournal(issn, nrows, offset)
  if res is not None:
    df = df.append(res)
    print(df['container-title'].iloc[-1])
  else:
    print('skipped')

2046-2069
['RSC Adv.']
2050-7488
['J. Mater. Chem. A']
1463-9076
['Phys. Chem. Chem. Phys.']
2050-7526
['Journal of Materials Chemistry C']
0004-9425
['Australian Journal of Chemistry']
0022-1430
['Journal of Glaciology']
2040-3364
['Nanoscale']
1359-7345
['Chem. Commun.']
0004-9506
['Australian Journal of Physics']
0009-8558
['Clay Minerals']
0021-8898
['Journal of Applied Crystallography']
1477-9226
['Dalton Trans.']


# Data Cleansing

In [0]:
newdf = pd.DataFrame()

In [0]:
newdf['DOI']= df.DOI
newdf['abstract'] = df.abstract
newdf['title'] = df.title.apply(lambda x: listToItem(x))
newdf['container-title'] = df['container-title'].apply(lambda x: listToItem(x))
newdf['is-referenced-by-count'] = df['is-referenced-by-count']
# newdf.head()
newdf['cleanTitle'] = newdf.title.apply(lambda x : cleandata(x))
newdf['cleanAbstract'] = newdf.abstract.apply(lambda x : cleandata(x))
newdf = newdf[['DOI', 'cleanTitle', 'container-title', 'cleanAbstract', 'is-referenced-by-count']]
newdf.head()

,DOI,cleanTitle,container-title,cleanAbstract,is-referenced-by-count
0,10.1039/c3ra46913a,esr study spin adducts three analogues depmpo ...,RSC Adv.,paverage geometries nitroxide adducts various ...,1
1,10.1039/c3ra46462h,reactions mono bicyclic enol ethers i–hydroper...,RSC Adv.,preactions mono bicyclic enol ethers isubsub–h...,7
2,10.1039/c3ra46915h,copper catalyzed oxidative cascade coupling n ...,RSC Adv.,pan oxidative cascade coupling reaction italic...,19
3,10.1039/c3ra46453a,catalytic wet peroxide oxidation phenol ch ch ...,RSC Adv.,punder ambient temperature atmospheric pressur...,10
4,10.1039/c3ra46967k,supramolecular analyte recognition experiment ...,RSC Adv.,psynergy experimental theoretical tools made s...,5


## Data Dump to Drive

In [0]:
import datetime
filename = 'data'+str(datetime.datetime.now())[:-7]+'.csv'
newdf.to_csv('gdrive\\' +filename)

In [0]:
!ls

'data2019-04-02 12:48:35.csv'  'gdrive\data2019-04-02 12:51:34.csv'
 gdrive			        sample_data


In [0]:
from google.colab import files

files.download('data2019-04-02 12:48:35.csv')

# NOT IN USE'

In [0]:
# Use
def doiToAbstract(doi):
    url = "http://api.crossref.org/works/"  + doi
    headers={"Accept": "application/json"}
    req = requests.get(url, headers=headers)
    result = req.json()
    if 'message' in result.keys():
      if 'abstract' in result['message']:
        return result['message']['abstract']
    else:
      return ""

In [0]:
def doiToCite(doi):
#     search_query = scholarly.search_pubs_query(doi)
#     print(doi)
#     result = next(search_query)
#     if hasattr(result, 'citedby'):
#         return result.citedby
#     else:
#         return 0
    return counts.citation_count(doi = doi)

In [0]:
url = "https://api.crossref.org/journals/0378-7788/works"
headers={"Accept": "application/json"}
req = requests.get(url, headers=headers)
json_str = json.dumps(req.json()['message']['items'])
df = pd.read_json(json_str)

In [0]:
df['container-title'] = df['container-title'].apply(lambda x: listToItem(x))
df['ISSN'] = df['ISSN'].apply(lambda x: listToItem(x))
df['container-title'].value_counts()

Energy and Buildings    20
Name: container-title, dtype: int64

In [0]:
# def getDfByJournal(doiPrefix, nrows):
#   url = "https://api.crossref.org/works?filter=has-abstract:true,prefix:"+doiPrefix+"&rows="+str(nrows)
#   headers={"Accept": "application/json"}
#   req = requests.get(url, headers=headers)
#   json_str = json.dumps(req.json()['message']['items'])
#   df = pd.read_json(json_str)
#   df.head()
#   return df

In [0]:
df_101016 = getDfByJournal("10.1016/j.enbuild" , 10)
df_101016.head()

,DOI,ISSN,URL,abstract,author,container-title,content-domain,created,deposited,indexed,...,prefix,published-print,publisher,reference-count,references-count,score,source,title,type,volume
0,10.2478/s11772-013-0089-3,[1896-3757],http://dx.doi.org/10.2478/s11772-013-0089-3,<jats:title>Abstract</jats:title><jats:p>Night...,"[{'given': 'K.', 'family': 'Chrzanowski', 'seq...",[Opto-Electronics Review],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2013, 3, 14]], 'date-time': '...","{'date-parts': [[2016, 12, 6]], 'date-time': '...","{'date-parts': [[2019, 2, 13]], 'date-time': '...",...,10.1016,"{'date-parts': [[2013, 1, 1]]}",Elsevier BV,1,1,1,Crossref,[Review of night vision technology],journal-article,21
1,10.2478/s11772-013-0081-y,[1896-3757],http://dx.doi.org/10.2478/s11772-013-0081-y,<jats:title>Abstract</jats:title><jats:p>Diffe...,"[{'given': 'A.', 'family': 'Sokół', 'sequence'...",[Opto-Electronics Review],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2013, 3, 14]], 'date-time': '...","{'date-parts': [[2017, 6, 21]], 'date-time': '...","{'date-parts': [[2019, 2, 13]], 'date-time': '...",...,10.1016,"{'date-parts': [[2013, 1, 1]]}",Elsevier BV,7,7,1,Crossref,[Thermal management of GaInNAs/GaAs VECSELs],journal-article,21
2,10.2478/s11772-013-0084-8,[1896-3757],http://dx.doi.org/10.2478/s11772-013-0084-8,<jats:title>Abstract</jats:title><jats:p>This ...,"[{'given': 'A.', 'family': 'Antończak', 'seque...",[Opto-Electronics Review],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2013, 3, 14]], 'date-time': '...","{'date-parts': [[2017, 6, 21]], 'date-time': '...","{'date-parts': [[2019, 2, 13]], 'date-time': '...",...,10.1016,"{'date-parts': [[2013, 1, 1]]}",Elsevier BV,10,10,1,Crossref,[Simple wavelength pre-selection and stabiliza...,journal-article,21
3,10.2478/s11772-013-0088-4,[1896-3757],http://dx.doi.org/10.2478/s11772-013-0088-4,<jats:title>Abstract</jats:title><jats:p>In th...,"[{'given': 'Y.', 'family': 'Chang', 'sequence'...",[Opto-Electronics Review],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2013, 3, 14]], 'date-time': '...","{'date-parts': [[2017, 6, 21]], 'date-time': '...","{'date-parts': [[2019, 2, 13]], 'date-time': '...",...,10.1016,"{'date-parts': [[2013, 1, 1]]}",Elsevier BV,28,28,1,Crossref,[A block-based watermarking scheme for image t...,journal-article,21
4,10.2478/s11772-013-0083-9,[1896-3757],http://dx.doi.org/10.2478/s11772-013-0083-9,<jats:title>Abstract</jats:title><jats:p>Defor...,"[{'given': 'G.', 'family': 'Derfel', 'sequence...",[Opto-Electronics Review],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2013, 3, 14]], 'date-time': '...","{'date-parts': [[2017, 6, 21]], 'date-time': '...","{'date-parts': [[2019, 2, 13]], 'date-time': '...",...,10.1016,"{'date-parts': [[2013, 1, 1]]}",Elsevier BV,26,26,1,Crossref,[Threshold voltages and optical retardation of...,journal-article,21


In [0]:
url = "http://api.crossref.org/works?query=building&filter=has-abstract:true&sort=is-referenced-by-count&rows=1000" 
headers={"Accept": "application/json"}
req = requests.get(url, headers=headers)
json_str = json.dumps(req.json()['message']['items'])
df = pd.read_json(json_str)
df.head()

,DOI,ISBN,ISSN,URL,abstract,alternative-id,archive,article-number,assertion,author,...,score,short-container-title,source,subject,subtitle,title,type,update-policy,update-to,volume
0,10.1039/c4cs00164h,NaN,"[0306-0012, 1460-4744]",http://dx.doi.org/10.1039/c4cs00164h,<p>Bio-inspired nano-materials can be formed b...,NaN,NaN,NaN,[{'value': 'This document is CrossCheck deposi...,"[{'given': 'Lihi', 'family': 'Adler-Abramovich...",...,5.511604,[Chem. Soc. Rev.],Crossref,NaN,NaN,[The physical properties of supramolecular pep...,journal-article,http://dx.doi.org/10.1039/rsc_crossmark_policy,NaN,43
1,10.1039/c4cs00002a,NaN,"[0306-0012, 1460-4744]",http://dx.doi.org/10.1039/c4cs00002a,<p>This review examines the governing factors ...,NaN,NaN,NaN,[{'value': 'This document is CrossCheck deposi...,"[{'given': 'C. K.', 'family': 'Brozek', 'seque...",...,5.511604,[Chem. Soc. Rev.],Crossref,NaN,NaN,[Cation exchange at the secondary building uni...,journal-article,http://dx.doi.org/10.1039/rsc_crossmark_policy,NaN,43
2,10.1107/s1399004714021683,NaN,[1399-0047],http://dx.doi.org/10.1107/s1399004714021683,<jats:p>The recent rapid development of single...,[S1399004714021683],NaN,NaN,[{'value': 'Acta Crystallographica Section D: ...,"[{'given': 'Alan', 'family': 'Brown', 'sequenc...",...,3.037635,"[Acta Cryst D Biol Crystallogr, Acta Cryst D B...",Crossref,NaN,NaN,[Tools for macromolecular model building and r...,journal-article,http://dx.doi.org/10.1107/cm_01,NaN,71
3,10.1039/c4cc00339j,NaN,"[1359-7345, 1364-548X]",http://dx.doi.org/10.1039/c4cc00339j,<p>This Feature article focuses on the use of ...,NaN,NaN,NaN,[{'value': 'This document is CrossCheck deposi...,"[{'given': 'Kasper S.', 'family': 'Pedersen', ...",...,6.408706,[Chem. Commun.],Crossref,NaN,NaN,[Single-molecule magnet engineering: building-...,journal-article,http://dx.doi.org/10.1039/rsc_crossmark_policy,NaN,50
4,10.1609/aimag.v31i3.2303,NaN,"[0738-4602, 0738-4602]",http://dx.doi.org/10.1609/aimag.v31i3.2303,<jats:p>IBM Research undertook a challenge to ...,NaN,NaN,NaN,NaN,"[{'given': 'David', 'family': 'Ferrucci', 'seq...",...,4.382310,[AIMag],Crossref,NaN,NaN,[Building Watson: An Overview of the DeepQA Pr...,journal-article,NaN,NaN,31


In [0]:
df['container-title'] = df['container-title'].apply(lambda x: listToItem(x))
df['ISSN'] = df['ISSN'].apply(lambda x: listToItem(x))

In [0]:
# issnList = list(df['ISSN'].value_counts().keys())
df = getDfByJournal(issnList[0], 1000)
# for issn in issnList[1:10]:
#   df.append(getDfByJournal(issn, 10))
df['container-title'].iloc[0]

['Construction Economics and Building']

In [0]:
for issn in issnList[1:10]:
  print(issn)
  df = df.append(getDfByJournal(issn, 1000))
  print(df['container-title'].iloc[-1])

1359-7345
['Chem. Commun.']
1477-9226
['Dalton Transactions']
1392-3730
['JOURNAL OF CIVIL ENGINEERING AND MANAGEMENT']
1662-8985
['Advanced Materials Research']
2046-2069
['RSC Adv.']
1662-7482
['Applied Mechanics and Materials']
0090-5992
['Nationalities Papers']
2050-7488
['J. Mater. Chem. A']
1684-9981
['Natural Hazards and Earth System Sciences']


In [0]:
df.to_csv('10J1000.csv')
files.download('10J1000.csv')

In [0]:
df2 = getDfByJournal(issnList[1], 10)

df2['container-title'].iloc[0]

['Chemical Communications']

In [0]:
df3 = getDfByJournal(issnList[2], 10)

df3['container-title'].iloc[0]

['Dalton Transactions']

In [0]:
def getDfByJournal(issn, nrows):
  url = "https://api.crossref.org/journals/"+issn+"/works?filter=has-abstract:true&rows="+str(nrows)
  headers={"Accept": "application/json"}
  req = requests.get(url, headers=headers)
  json_str = json.dumps(req.json()['message']['items'])
  df = pd.read_json(json_str)
  df.head()
  return df

In [0]:
df1 = getDfByJournal("2204-9029", 10)

In [0]:
df_5Journals = dfList[0]
for df in dfList[1:]:
  df_5Journals.append(df)
df_5Journals.tail()

,DOI,ISSN,URL,abstract,author,container-title,content-domain,created,deposited,indexed,...,reference,reference-count,references-count,relation,score,short-container-title,source,title,type,volume
393,10.5130/ajceb.v14i3.4158,[2204-9029],http://dx.doi.org/10.5130/ajceb.v14i3.4158,<jats:p>A book review of “Best Practice in Con...,"[{'given': 'Philip', 'family': 'Chan', 'sequen...",[Construction Economics and Building],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2014, 10, 1]], 'date-time': '...","{'date-parts': [[2018, 10, 11]], 'date-time': ...","{'date-parts': [[2019, 3, 6]], 'date-time': '2...",...,NaN,0,0,NaN,1,[CEB],Crossref,[A book review of “Best Practice in Constructi...,journal-article,14
394,10.5130/ajceb.v9i1.3011,[2204-9029],http://dx.doi.org/10.5130/ajceb.v9i1.3011,<jats:p>Public-private partnership (PPP) proje...,"[{'given': 'Xioa-Hua', 'family': 'Jin', 'seque...",[Construction Economics and Building],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2014, 10, 1]], 'date-time': '...","{'date-parts': [[2018, 10, 8]], 'date-time': '...","{'date-parts': [[2019, 3, 22]], 'date-time': '...",...,NaN,0,0,NaN,1,[CEB],Crossref,[Allocating Risks in Public-Private Partnershi...,journal-article,9
395,10.5130/ajceb.v11i3.2174,[2204-9029],http://dx.doi.org/10.5130/ajceb.v11i3.2174,<jats:p>New housing supply in Australia has be...,"[{'given': 'Junxiao', 'family': 'Liu', 'sequen...",[Construction Economics and Building],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2014, 10, 1]], 'date-time': '...","{'date-parts': [[2018, 10, 8]], 'date-time': '...","{'date-parts': [[2019, 3, 27]], 'date-time': '...",...,NaN,0,0,NaN,1,[CEB],Crossref,[Analysing the Relationship between New Housin...,journal-article,11
396,10.5130/ajceb.v16i3.5178,[2204-9029],http://dx.doi.org/10.5130/ajceb.v16i3.5178,<jats:p>Innovation is key for productivity imp...,"[{'given': 'Peter', 'family': 'Davis', 'sequen...",[Construction Economics and Building],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2016, 10, 26]], 'date-time': ...","{'date-parts': [[2018, 10, 8]], 'date-time': '...","{'date-parts': [[2019, 3, 29]], 'date-time': '...",...,NaN,0,0,NaN,1,[CEB],Crossref,[Assessing construction innovation: theoretica...,journal-article,16
397,10.5130/ajceb.v13i3.3280,[2204-9029],http://dx.doi.org/10.5130/ajceb.v13i3.3280,"<jats:p>CorporateSocial Responsibility (CSR), ...","[{'given': 'Lisa Robyn', 'family': 'Barnes', '...",[Construction Economics and Building],"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2014, 10, 1]], 'date-time': '...","{'date-parts': [[2018, 10, 8]], 'date-time': '...","{'date-parts': [[2019, 3, 30]], 'date-time': '...",...,NaN,0,0,NaN,1,[CEB],Crossref,[The relevance of the ISO26000 social responsi...,journal-article,13


In [0]:
df_5Journals["is-referenced-by-count"].value_counts()

0     169
1      70
2      40
3      36
4      20
5      14
6      13
7       8
8       6
9       5
11      4
12      3
14      3
28      1
10      1
15      1
16      1
20      1
23      1
82      1
Name: is-referenced-by-count, dtype: int64

In [0]:
df_5Journals.iloc[0]

DOI                                                10.5130/ajceb.v14i3.4154
ISSN                                                            [2204-9029]
URL                              http://dx.doi.org/10.5130/ajceb.v14i3.4154
abstract                  <jats:p>The purpose of this paper is twofold. ...
author                    [{'given': 'M. Reza', 'family': 'Hosseini', 's...
container-title                       [Construction Economics and Building]
content-domain               {'domain': [], 'crossmark-restriction': False}
created                   {'date-parts': [[2014, 10, 1]], 'date-time': '...
deposited                 {'date-parts': [[2018, 10, 8]], 'date-time': '...
indexed                   {'date-parts': [[2019, 2, 14]], 'date-time': '...
is-referenced-by-count                                                    0
issn-type                    [{'value': '2204-9029', 'type': 'electronic'}]
issue                                                                     3
issued      

In [0]:


# Edit current section for enable local dump 

# df_5Journals.to_csv('5journals_5000.csv')
# files.download('5journals_5000.csv')